### Lagrangian v.s. Eulerian: Two Views of Continuums
1. Lagrangian View, 拉格朗日视角:
    - Sensors that move passively with the simulated material(随波逐流)
    - 粒子
2. Eulerian View, 欧拉视角:
    - Still sensors that never moves.(岿然不动)
    - 网格

### Mass-Spring System, 弹簧-质点模型
- Extremely simple
- But very useful!
    - Cloth
    - Elastic objects
    - ...

### Mathematical Model
\begin{equation*}
\begin{array}{**rl**}
    {\rm f}_ {ij} &=-k(||{\rm x}_ {i}-{\rm x}_ {j}||_ {2}- l_{ij})(\widehat{{\rm x}_ {i}-{\rm x}_ {j}})~~~~(Hooke's Law) \\
    {\rm f}_ {i}  &= \displaystyle\sum_{j}^{j\ne i}{\rm f}_ {ij}   \\
    \displaystyle\frac{\partial {\rm v}}{\partial t}  &= \displaystyle\frac{1}{m_{i}}{\rm f}_ {i}
\end{array}
\end{equation*}

### 雅可比迭代法
对于矩阵 $Ax=b$, $A$非奇异，且对角元不为0，可以将原方程组改写为：

\begin{equation*}
\left\{  
             \begin{array}{**lr**}  
             x_{1}=\displaystyle\frac{1}{a_{11}}\left(b_{1}-a_{11}x_{2}-...-a_{1n}x_{n}\right), &  \\  
             x_{2}=\displaystyle\frac{1}{a_{22}}\left(b_{2}-a_{21}x_{1}-...-a_{2n}x_{n}\right), & \\ 
             ...... & \\ 
             x_{n}=\displaystyle\frac{1}{a_{nn}}\left(b_{n}-a_{n1}x_{1}-...-a_{(n,n-1)}x_{n-1}\right), &   
             \end{array}  
\right.
\end{equation*}

In [2]:
import taichi as ti
import random

ti.init(arch=ti.cpu)

n = 20

A = ti.field(dtype=ti.f32, shape=(n, n))
x = ti.field(dtype=ti.f32, shape=n)
new_x = ti.field(dtype=ti.f32, shape=n)
b = ti.field(dtype=ti.f32, shape=n)


# 单步雅可比迭代
@ti.kernel
def iterate():
    for i in range(n):
        r = b[i]
        for j in range(n):
            if i != j:
                r -= A[i, j] * x[j]

        new_x[i] = r / A[i, i]

    for i in range(n):
        x[i] = new_x[i]


# 计算误差
@ti.kernel
def residual() -> ti.f32:
    res = 0.0

    for i in range(n):
        r = b[i] * 1.0
        for j in range(n):
            r -= A[i, j] * x[j]
        res += r * r

    return res


for i in range(n):
    for j in range(n):
        A[i, j] = random.random() - 0.5

    A[i, i] += n * 0.1

    b[i] = random.random() * 100

for i in range(100):
    iterate()
    print(f'{i}, residual={residual():0.10f}')

for i in range(n):
    lhs = 0.0
    for j in range(n):
        lhs += A[i, j] * x[j]
    assert abs(lhs - b[i]) < 1e-4

[Taichi] Starting on arch=x64
[Taichi] materializing...
0, residual=24736.9296875000
1, residual=3674.2048339844
2, residual=1047.6331787109
3, residual=479.8469848633
4, residual=127.7566528320
5, residual=38.9668617249
6, residual=14.7183237076
7, residual=7.2304415703
8, residual=3.6475548744
9, residual=2.1913433075
10, residual=0.9774737954
11, residual=0.5886521935
12, residual=0.2952284813
13, residual=0.1701266766
14, residual=0.0896255150
15, residual=0.0499186404
16, residual=0.0264424756
17, residual=0.0149449417
18, residual=0.0079610366
19, residual=0.0044234050
20, residual=0.0023861893
21, residual=0.0013163054
22, residual=0.0007145723
23, residual=0.0003932702
24, residual=0.0002135455
25, residual=0.0001171976
26, residual=0.0000639685
27, residual=0.0000350286
28, residual=0.0000190337
29, residual=0.0000104412
30, residual=0.0000057530
31, residual=0.0000031255
32, residual=0.0000016539
33, residual=0.0000009213
34, residual=0.0000005376
35, residual=0.0000002782
36

In [2]:
import taichi as ti

ti.init(arch=ti.gpu)

a = ti.field(dtype=ti.f32, shape=1)

print("hello world")


[Taichi] Starting on arch=cuda
hello world


$$ y = a + b x $$